<a href="https://colab.research.google.com/github/gabo-CW/Monitoring_test/blob/main/Task2-Solve%20the%20problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import sqlite3
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
conn = sqlite3.connect('case1_transactions.db')

In [6]:
df1 = pd.read_csv('transactions_1.csv')
df2 = pd.read_csv('transactions_2.csv')

In [ ]:
df1.to_sql('tabela1', conn, if_exists='replace', index=False)
df2.to_sql('tabela2', conn, if_exists='replace', index=False)

In [8]:
conn = sqlite3.connect('case1_transactions.db')

In [9]:
query = "SELECT * FROM tabela1"
query2 = "SELECT * FROM tabela2"

In [10]:
df_resultado = pd.read_sql_query(query, conn)
df_resultado2 = pd.read_sql_query(query2, conn)

In [ ]:
print(df_resultado)
print("----------------------------------------------------------------------------")
print(df_resultado2)

In [ ]:
for status in df1['status'].unique():
    subset = df1[df1['status'] == status]
    plt.plot(subset['time'], subset['f0_'], label=status)

# Preparing the first dataframe to analise
time_status_combinations = pd.MultiIndex.from_product([df1['time'].unique(), df1['status'].unique()], names=['time', 'status'])
df1_full = df1.set_index(['time', 'status']).reindex(time_status_combinations, fill_value=0).reset_index()

tab = px.line(df1_full, x='time', y='f0_', color='status', markers=True,
              title='Transações por Status ao longo do Tempo df1')

tab.update_layout(
    xaxis_title='Time',
    yaxis_title='Transactions',
    hovermode='x unified',  # Show all the informations on X
)

In [ ]:
for status in df1['status'].unique():
    subset = df1[df1['status'] == status]
    plt.plot(subset['time'], subset['f0_'], label=status)

# Preparing the second dataframe to analise
time_status_combinations = pd.MultiIndex.from_product([df2['time'].unique(), df2['status'].unique()], names=['time', 'status'])
df2_full = df2.set_index(['time', 'status']).reindex(time_status_combinations, fill_value=0).reset_index()

tab = px.line(df2_full, x='time', y='count', color='status', markers=True,
              title='Transações por Status ao longo do Tempo df2')

tab.update_layout(
    xaxis_title='Time',
    yaxis_title='Transactions',
    hovermode='x unified',  # Mostra todas as informações em um único hover
)

In [ ]:
# avg_by_status_df1 = df1.groupby('status')['f0_'].mean().round(2)
# outlier_low = df1.groupby('status')['f0_'].quantile(0.2)
# outlier_high = df1.groupby('status')['f0_'].quantile(0.8)
# avg_without_outliers = df1.groupby('status').apply(lambda x: x[(x['f0_'] >= outlier_low[x.name]) & (x['f0_'] <= outlier_high[x.name])]['f0_'].mean()).round(2)
# print(avg_without_outliers)

# print('--------------------------')

mean_by_status = df1.groupby('status')['f0_'].mean()
std_by_status = df1.groupby('status')['f0_'].std()

avg_without_outliers_df1 = df1.groupby('status').apply(
    lambda x: x[(x['f0_'] >= (mean_by_status[x.name] - 2 * std_by_status[x.name])) &
                (x['f0_'] <= (mean_by_status[x.name] + 2 * std_by_status[x.name]))]['f0_'].mean()
).round(2)
print('Excluindo outliers baseado no desvio padrão', avg_without_outliers_df1)

# print('--------------------------')


# avg_by_status_df1 = df1.groupby('status')['f0_'].mean().round(2)
# print(avg_by_status_df1)

In [ ]:
mean_by_status = df2.groupby('status')['count'].mean()
std_by_status = df2.groupby('status')['count'].std()

avg_without_outliers_df2 = df2.groupby('status').apply(
    lambda x: x[(x['count'] >= (mean_by_status[x.name] - 2 * std_by_status[x.name])) &
                (x['count'] <= (mean_by_status[x.name] + 2 * std_by_status[x.name]))]['count'].mean()
).round(2)
print('Excluindo outliers baseado no desvio padrão', avg_without_outliers_df2)

In [16]:
# Function about check if the set of transactions requires an alert
def check_anomaly(transaction_time, transaction_status, transaction_count, mean_values):
    if transaction_status == 'approved':
      return None
    if transaction_status in mean_values and (transaction_status == 'denied' or transaction_status == 'reversed' or transaction_status == 'failed' or transaction_status == 'backend_reversed'):
        mean_value = mean_values[transaction_status]
        # The rule for the alert is to be the twice of average
        if transaction_count > mean_value * 2:
            return f"{transaction_time} High number of {transaction_status} transactions ({transaction_count})"
    return None

In [17]:
# Check_anomaly test
sample_df1 = df1[['time', 'status', 'f0_']].apply(lambda row: check_anomaly(row['time'], row['status'], row['f0_'], avg_without_outliers_df1), axis=1)
sample_df2 = df2[['time', 'status', 'count']].apply(lambda row: check_anomaly(row['time'], row['status'], row['count'], avg_without_outliers_df2), axis=1)

In [ ]:
alerts_df1 = sample_df1.dropna()
alerts_df2 = sample_df2.dropna()

print('Transactions_1\n', alerts_df1, '\n')
print('Transactions_2\n', alerts_df2, '\n')

In [ ]:
# Make anomalies dataframes
anomalies_df1 = df1[df1.apply(lambda row: check_anomaly(row['time'], row['status'], row['f0_'], avg_without_outliers_df1) is not None, axis=1)]
anomalies_df2 = df2[df2.apply(lambda row: check_anomaly(row['time'], row['status'], row['count'], avg_without_outliers_df2) is not None, axis=1)]

# Make static anomalies chart
fig1 = px.scatter(anomalies_df1, x='time', y='f0_', color='status', title='Anomalias nas Transações (df1)',
                  labels={'f0_': 'Número de Transações', 'time': 'Tempo'})
fig1.update_layout(xaxis_title='Tempo', yaxis_title='Número de Transações')

fig2 = px.scatter(anomalies_df2, x='time', y='count', color='status', title='Anomalias nas Transações (df2)',
                  labels={'count': 'Número de Transações', 'time': 'Tempo'})
fig2.update_layout(xaxis_title='Tempo', yaxis_title='Número de Transações')

fig1.show()
fig2.show()

In [28]:
from google.colab import userdata
secret_key = userdata.get('ngrok_auth')

In [ ]:
!pip install flask
!pip install pyngrok
!ngrok authtoken {secret_key}

In [30]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

app = Flask(__name__)

@app.route('/check_transaction', methods=['POST'])
def check_transaction():
    transaction_data = request.get_json()

    status = transaction_data.get('status')
    count = transaction_data.get('count')
    transaction_time = transaction_data.get('time')

    # Check the anomalies
    alert_message = check_anomaly(transaction_time, status, count, avg_without_outliers_df2) #or check_anomaly(transaction_time, status, count, avg_without_outliers_df2)

    if alert_message:
        return jsonify({"alert": alert_message}), 200
    return jsonify({"message": "No anomalies detected"}), 204
    # return '', 204


def run_flask():
    # Start ngrok tunnel for port 8000
    public_url = ngrok.connect(8000)
    print(f" * ngrok tunnel available at {public_url}")

    app.run(port=8000)

# Using a thread to run in the background
thread = threading.Thread(target=run_flask)
thread.start()

In [ ]:
import requests
ngrokurl = "https://b4a4-35-221-247-93.ngrok-free.app/check_transaction"  # URL from ngrok

# Function to send transactions to Flask endpoint and show the result
def send_transactions(df, status_col, count_col):
    for index, row in df.iterrows():
        transaction_data = {
            "status": row[status_col],
            "count": row[count_col]
        }
        response = requests.post(ngrokurl, json=transaction_data)

        # Check the status code
        if response.status_code == 200:
            result = response.json()
            print(f"Transaction {index+1}: {result}")
        elif response.status_code == 204:
          # continue
            print(f"Transaction {index+1}: No anomalies detected")
        else:
            # continue
            print(f"Transaction {index+1}: Unexpected response status {response.status_code}")

print("Testing transactions from file 1...")
send_transactions(df1, 'status', 'f0_')

print("Testing transactions from file 2...")
send_transactions(df2, 'status', 'count')

# Check the static response just to test, delete after
# response = requests.post(ngrokurl, json={"status": "denied", "count": 1033})
# result = response.json()

print(response.json())